Проект сделан на основе выработанного шаблона:
- https://github.com/evgeny-ydsc/first-project/blob/main/шаблон%20проекта/шаблон%20проекта.ipynb

# Название проекта

краткое описание проекта

# Постановка задачи

## Критерии успеха
- ...

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных



- **Целевой признак:**


# Инициализация

## Импорт библиотек

In [46]:
CLASSIFICATION_TASK = 1
REGRESSION_TASK = 0

In [47]:
!pip install phik -q
import phik

In [ ]:
if CLASSIFICATION_TASK:
    !pip install lightgbm -q
    import lightgbm as lgb

In [ ]:
if CLASSIFICATION_TASK:
    !pip install category_encoders -q
    from category_encoders import TargetEncoder

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt 
import seaborn as sns

if REGRESSION_TASK:
    from sklearn.linear_model import LinearRegression
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.svm import SVR 
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import (root_mean_squared_error)
if CLASSIFICATION_TASK:
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.svm import SVC 
    from sklearn.metrics import classification_report, roc_curve, auc, \
                                RocCurveDisplay, 
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay, \
                                roc_curve, precision_recall_curve, roc_auc_score
                                

from sklearn.model_selection import train_test_split, \
                                RandomizedSearchCV #, GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (OneHotEncoder, OrdinalEncoder, 
                                   MinMaxScaler, StandardScaler)
from sklearn.compose import ColumnTransformer

import re
import time
from datetime import datetime, timedelta

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN

## Установка главных параметров

In [51]:
RANDOM_STATE = 42

# Вспомогательные функции

In [ ]:
def correct_path (csv_name, folder=""):
    if not os.path.exists(folder + csv_name):
        return '/datasets/' + csv_name
    return folder + csv_name

def def_load_csv(csv, folder="", print_info=True):
    display(f"⬇-------- Таблица {csv}: ----------⬇")
    df = pd.read_csv(correct_path(csv, folder))
    if print_info:
        display(df.info())
        display(df.head())
    return df

def columns_to_snake_case (df):
    names = {}
    for c in df.columns:
        names [c] = re.sub(r'(?<!^)(?=[A-Z])', '_', c).lower() \
            .translate(str.maketrans('','',' (){}[]<>/\\'))
    df.rename(columns=names, inplace=True)
    return df

# визуализирует все категориальные фичи датафрейма в виде пирогов
# визуализирует все категориальные фичи датафрейма в виде ящиков с усами и гистограмм с распределением, выводит descirbe
# поддерживает любое количество фильтров, фильтры поочерёдно применяются к датафрейму как запрос в query(),
#   графики выводятся для каждого фильтра
#   чтобы вывести графики без фильтра нужно передать пустой запрос, или же не передавать фильтры вовсе


def keep_top_group_other (d, top_n = 10):
    vc = d.value_counts()
    l = len (vc)
    if l > top_n:
        t = vc[:top_n].index
        display(t)
        d = d.apply(lambda x: x if x in t else f'Other {l-top_n} categories')
    return d

def do_eda(df, columns=[], filters=[]):
    bins=25
    n_filters = len(filters)
    PIE_WIDTH = 5
    
    if len (columns) == 0:
        columns = df.columns

    # вывод графиков по категориальным фичам 
    for column in columns:
        if df[column].dtype == 'O':
            display (f"⬇-------- столбец {column} --------⬇")
            title0 = f"столбец {column}, количество и доля значений"
            df_show = df[column]
            if n_filters > 0 and filters[0] != '':
                title = title0 + f"\nфильтр: {filters[0]}"
                df_show = df.query(filters[0])[column]
            # if n_filters ...: 👷🚩[todo] поддержать фильтры при помощи гистограммы с группировкой
            v = keep_top_group_other(df_show) \
                .value_counts(normalize=False, ascending=True)
            ax = v.plot.barh(title=title0);
            s = v.values.sum()
            for i, value in enumerate(v.values):
                ax.text(value + 5, i, f'{value} ({int(value*100/s)}%)')
            xmin, xmax = ax.get_xlim()
            ax.set_xlim(xmin, 1.2*xmax)
        plt.show();

    # вывод графиков по числовым фичам 
    if n_filters == 0:
        filters = ['']
        n_filters = 1
    for column in columns:
        if df[column].dtype in ['int64', 'float64'] and column != "id":
            display (f"⬇-------- Следующий признак: {column} --------⬇")
            fig, axs = plt.subplots(nrows=2*max(1, n_filters), ncols=1, 
                                    figsize=(6, 7*n_filters))
            for i in range (n_filters):
                title = f"столбец {column}"
                df_show = df[column]
                if filters[i] != '':
                    title = title + f", фильтр: {filters[i]}"
                    df_show = df.query(filters[i])[column]
                axs[i*2].boxplot(df_show, vert=False);
                ##v1 sns.boxplot(data=df_show, ax=axs[i*2], palette=['lightsteelblue'], orient='h')
                axs[i*2].set_title(title, fontsize=9)
                sns.histplot(df_show, bins=bins, kde=True, ax=axs[i*2+1], alpha=0.8)
                axs[i*2+1].set_xlabel('')
                display(f"фильтр = '{filters[i]}'")
                display(df_show.describe())
            plt.show();

def print_unique_text_features(df):
    for c in df.columns:
        if df[c].dtype == 'O':
            x = df[c].unique()
            x = np.sort(x[~pd.isnull(x)])
            display(f"⬇-------- Feature \"{c}\"--------⬇")
            display(x)

# example:
# df = split_date (df, 'created_at', needed_parts='ymdwh', format='%Y-%m-%d %H:%M:%S'),
# use it after solving with NaNs to prevent appearing floats
def split_date (df, date_column, needed_parts, format="", drop_date_column=False):
    if format != "":
        df[date_column] = pd.to_datetime(df[date_column], format=format)
    if ['y' in needed_parts]:
        df [date_column + "_year"] = df[date_column].dt.year.astype(int)
    if ['m' in needed_parts]:
        df [date_column + "_month"] = df[date_column].dt.month.astype(int)
    if ['d' in needed_parts]:
        df [date_column + "_day"] = df[date_column].dt.day.astype(int)
    if ['w' in needed_parts]:
        df [date_column + "_weekday"] = df[date_column].dt.weekday.astype(int)
    if ['h' in needed_parts]:
        df [date_column + "_hour"] = df[date_column].dt.hour.astype(int)
    if drop_date_column:
        df = df.drop([date_column], axis=1)
    return df

def show_nans (df):
    n = df.isna().sum()
    l = len(df)
    display("empty parts:")
    display(round(n [n>0] / l, 2))

def get_multicollinear_features (phik_matrix):
    mc_features=[]
    while True:
        if len(phik_matrix) == 0:
            break
        mc_features_ranked = phik_matrix[(phik_matrix >= 0.9) & (phik_matrix < 1)].sum()
        mc_features_ranked = mc_features_ranked[mc_features_ranked>0] \
            .sort_values(ascending=False)
        if len(mc_features_ranked) == 0:
            break
        top_mc_feature = mc_features_ranked.index[0]
        phik_matrix = phik_matrix.drop(top_mc_feature,axis=0)
        phik_matrix = phik_matrix.drop(top_mc_feature,axis=1)
        mc_features.append(top_mc_feature)
    return mc_features, phik_matrix

# Подготовка данных

## Загрузка таблиц, исправление сепараторов

In [53]:
#df = def_load_csv ('....csv')

## Поправим имена полей

In [54]:
'''
df = columns_to_snake_case(df)
display (df.info())
display (df.head())
'''

'\ndf = columns_to_snake_case(df)\ndisplay (df.info())\ndisplay (df.head())\n'

## Исправим типы данных

In [55]:
#df['...'] = pd.to_datetime( df['...'], format='%Y-%m-%d %H:%M:%S' )
#df['...'] = df['...'].astype('object')
#df['..._year'] = df['...'].dt.year
#df['..._month'] = df['...'].dt.month

In [56]:
'''
display(df.info())
display(df.head())
'''

'\ndisplay(df.info())\ndisplay(df.head())\n'

## Удалим поля, приводящие к утечке данных или заведомо бесполезные

- `...` - ... комментарий

In [57]:
#df = df.drop(['...', '...'], axis=1)

## Изучим пропуски

In [ ]:
'''
show_nans (df)
'''

'\nn = df.isna().sum()\nl = len(df)\nround(n [n>0] / l, 2)\n'

In [59]:
'''
df['vehicle_type'] = df['vehicle_type'] \
    .fillna(df.groupby(['brand', 'model'])['vehicle_type'] \
            .transform(lambda x: x.mode()[0] if not x.mode().empty else None))
'''

"\ndf['vehicle_type'] = df['vehicle_type']     .fillna(df.groupby(['brand', 'model'])['vehicle_type']             .transform(lambda x: x.mode()[0] if not x.mode().empty else None))\n"

👷🚩[todo] если же какие-то пропуски всё-таки останутся - избавимся от них с помощью imputer-ов в pipeline

## Изучим и удалим явные дубликаты в датафрейме

In [60]:
'''
display(df.duplicated().sum())
df = df.drop_duplicates()
'''

'\ndisplay(df.duplicated().sum())\ndf = df.drop_duplicates()\n'

## Изучим неявные дубликаты в датафрейме


In [61]:
#print_unique_text_features(df)

Заменим значения xxx фичи f на yyy.

In [ ]:
#df.loc[df['f']=='xxx', 'f'] = 'yyy'

## Сравним соответствующие фичи в тренировочной и тестовой выборках если они из разных источников, а не из одной таблицы(состав, типы, значения) 


# Разобъём выборку на тренировочную и тестовую (со стратификацией по целевому признаку в случае классификации)

In [63]:
'''
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    random_state=RANDOM_STATE
)
'''

"\nX = df.drop(['price'], axis=1)\ny = df['price']\n\nX_train, X_test, y_train, y_test = train_test_split(\n    X, \n    y, \n    random_state=RANDOM_STATE\n)\n"

# Исследовательский анализ данных (EDA)

In [64]:
#do_eda(X_train)

Оценим целевой признак:

In [65]:
#do_eda(pd.DataFrame({"y_train": y_train}))

## Выводы и задачи по итогам EDA
1. ...
    1. 👷🚩[todo] ...

Скорректируем данные, заменив аномалии:

In [66]:
'''
X_train.loc[(X_train['xxx'] <10) | (X_train['xxx'] > 1600), 'xxx'] = \
    X_train.groupby(['brand', 'model'])['power'] \
        .transform(lambda x: x.mode()[0] if not x.mode().empty else None)
'''

"\nX_train.loc[(X_train['xxx'] <10) | (X_train['xxx'] > 1600), 'xxx'] =     X_train.groupby(['brand', 'model'])['power']         .transform(lambda x: x.mode()[0] if not x.mode().empty else None)\n"

Скорректируем данные, удалив аномалии:

In [67]:
#X_train = X_train.query('...')

Удалим ненужные столбцы:

In [68]:
'''
columns_to_delete = ['...', '...']
X_train = X_train.drop(columns_to_delete, axis=1)
X_test = X_test.drop(columns_to_delete, axis=1)
'''

"\ncolumns_to_delete = ['...', '...']\nX_train = X_train.drop(columns_to_delete, axis=1)\nX_test = X_test.drop(columns_to_delete, axis=1)\n"

## Оценим графики после удалений:

In [69]:
#do_eda(X_train)

## Удалим из y_train те же строки, что удалили в процессе из X_train и сбросим индексы

In [70]:
'''
y_train = y_train[X_train.index]
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
'''

'\ny_train = y_train[X_train.index]\nX_train = X_train.reset_index(drop=True)\ny_train = y_train.reset_index(drop=True)\n'

# Проверим мультиколлинеарность

In [ ]:
'''
interval_cols=['xxx', 'xxx', 'xxx', 'xxx']
   
bins = {}
for c in interval_cols:
    bins[c] = 10;

_, ax = plt.subplots(figsize=(25,25))
ph_mx = X_train.phik_matrix(interval_cols=interval_cols, bins=bins)
sns.heatmap(ph_mx, annot=True, fmt='.2f', ax=ax);

columns_to_delete, ph_mx_no_mc = get_multicollinear_features (ph_mx)
display ("Мультиколлинеарные фичи для удаления:")
display (columns_to_delete)

'''

"\n_, ax = plt.subplots(figsize=(25,25))\nph_mx = X_train.phik_matrix(interval_cols=['kilometer', 'power', 'registration_year'],\n                bins={'kilometer':10, 'power':10, 'registration_year':10})\nsns.heatmap(ph_mx, annot=True, fmt='.2f', ax=ax);\n"

*Как правило, о мультиколлинеарности говорят при значении коэффициента корреляции от 0.9 до 0.95 по модулю. В такой ситуации лишние коррелирующие признаки нужно удалять из обучающей выборки.*

In [73]:
'''
n = len(ph_mx.columns)
for i in range(1,n):
    for j in range(0,i):
        if ph_mx.iloc[i][j] >= 0.9:
            display (f'{ph_mx.columns[i]} <-> {ph_mx.columns[j]}')           
'''

"\nn = len(ph_mx.columns)\nfor i in range(1,n):\n    for j in range(0,i):\n        if ph_mx.iloc[i][j] >= 0.9:\n            display (f'{ph_mx.columns[i]} <-> {ph_mx.columns[j]}')           \n"

Удалим ненужные столбцы:

In [74]:
'''
columns_to_delete = ['...', '...']
X_train = X_train.drop(columns_to_delete, axis=1)
X_test = X_test.drop(columns_to_delete, axis=1)
'''

"\ncolumns_to_delete = ['...', '...']\nX_train = X_train.drop(columns_to_delete, axis=1)\nX_test = X_test.drop(columns_to_delete, axis=1)\n"

In [75]:
#display(X_train.info())
#display(X_train.head())

# Проверим сбалансированность тренировочной выборки относительно целевого класса (в случае классификации)

In [ ]:
#y_train.value_counts(normalize=True)

# Обучение моделей

## Pipeline

In [76]:
#cat_columns = ['...', '...']

In [77]:
'''
for c in X_train.columns:
    if X_train[c].dtype == 'O':
        X_train[c] = X_train[c].astype("category")
display(X_train.info())
'''

'\nfor c in X_train.columns:\n    if X_train[c].dtype == \'O\':\n        X_train[c] = X_train[c].astype("category")\ndisplay(X_train.info())\n'

In [78]:
#num_columns = ['...', '...']

In [79]:
'''
cat_pipe = Pipeline(
    [('simpleImputer_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
    ]
    )

data_preprocessor = ColumnTransformer(
    [
        ('ohe', cat_pipe, cat_columns),
        ('num', StandardScaler(), num_columns),
    ], 
    remainder='passthrough'
) 
'''

"\ncat_pipe = Pipeline(\n    [('simpleImputer_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),\n     ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))\n    ]\n    )\n\ndata_preprocessor = ColumnTransformer(\n    [\n        ('ohe', cat_pipe, cat_columns),\n        ('num', StandardScaler(), num_columns),\n    ], \n    remainder='passthrough'\n) \n"

In [80]:
'''
%%time

pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
]
)

param_grid = [
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
        'models__max_depth': [4,7],
        'models__max_features': [10, 50],
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
    }
    ,
    {
        'models': [RandomForestClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': [10, 20],
        'models__min_samples_split': [5, 10],
        'models__min_samples_leaf': [2, 4],
        'models__bootstrap': [True, False],
        'preprocessor__num': [MinMaxScaler()]   
    }
    ,
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear', 
            penalty='l1',
            class_weight='balanced'
        )],
        'models__C': range(1, 3),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
    }
    ,
    {
        'models': [SVC(
            random_state=RANDOM_STATE, 
            kernel='linear', probability=True, class_weight='balanced'
        )],
        'models__kernel': ['poly', 'rbf'],
        'models__degree': [2, 5, 10],
        'models__C': range(2, 3),
    }
]

search = RandomizedSearchCV(
    pipe_final, 
    param_grid,
    cv=3,
    scoring='f1',
    n_iter=30,
    verbose = 10,
    n_jobs=-1
)

search.fit(X_train, y_train)

grid_search.best_params_

best_model = search.best_estimator_
'''

"\n%%time\n\npipe_final = Pipeline([\n    ('preprocessor', data_preprocessor),\n    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))\n]\n)\n\nparam_grid = [\n    {\n        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced')],\n        'models__max_depth': [4,7],\n        'models__max_features': [10, 50],\n        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   \n    }\n    ,\n    {\n        'models': [RandomForestClassifier(random_state=RANDOM_STATE)],\n        'models__max_depth': [10, 20],\n        'models__min_samples_split': [5, 10],\n        'models__min_samples_leaf': [2, 4],\n        'models__bootstrap': [True, False],\n        'preprocessor__num': [MinMaxScaler()]   \n    }\n    ,\n    {\n        'models': [LogisticRegression(\n            random_state=RANDOM_STATE, \n            solver='liblinear', \n            penalty='l1',\n            class_weight='balanced'\n        )],\n        'models__C': range(1, 3),\n        '

# Анализ результатов

## Метрики качества на валидационной выборке

In [81]:
#display ('Метрика качества лучшей модели на кроссвалидации: ', search.best_score_)

## Метрики качества на тестовой выборке

In [82]:
# оценка классификации
'''
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
'''

'\ny_pred = best_model.predict(X_test)\nprint(classification_report(y_test, y_pred))\n'

In [83]:
# оценка регрессии
'''
y_pred = best_model.predict(X_test)
display ('RMSE на тестовой выборке = ', root_mean_squared_error(y_test, y_pred))
'''

"\ny_pred = best_model.predict(X_test)\ndisplay ('RMSE на тестовой выборке = ', root_mean_squared_error(y_test, y_pred))\n"

In [ ]:
# ROC Curve
'''
RocCurveDisplay.from_estimator(model, X_test, y_test);
'''

In [ ]:
# PrecisionRecallDisplay
'''
RocCurveDisplay.from_estimator(model, X_test, y_test);
'''

## Анализ остатков (в случае регрессии)

In [84]:
'''
residuals = y_test - y_pred

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
axes[0].hist(residuals, bins=33)
axes[0].set_title('Гистограмма распределения остатков')
axes[0].set_xlabel('Остатки')

axes[1].scatter(predictions, residuals)
axes[1].set_xlabel('Предсказания модели')
axes[1].set_ylabel('Остатки')
axes[1].set_title('Анализ дисперсии')
plt.show()
'''

"\nresiduals = y_test - y_pred\n\nfig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))\naxes[0].hist(residuals, bins=33)\naxes[0].set_title('Гистограмма распределения остатков')\naxes[0].set_xlabel('Остатки')\n\naxes[1].scatter(predictions, residuals)\naxes[1].set_xlabel('Предсказания модели')\naxes[1].set_ylabel('Остатки')\naxes[1].set_title('Анализ дисперсии')\nplt.show()\n"

Для всех случаев ненормального распределения данных подходят два совета:
- Найти и добавить в модель дополнительные признаки. Скорее всего, на этапе сбора данных не была учтена какая-то информация.
- Трансформировать данные, чтобы изменить взаимосвязь входных признаков с целевым. Этим преобразованиям вы научитесь в следующем

## Анализ важности фичей

In [85]:
'''
samples = 20
 
# Извлечение лучшей модели из результатов RandomizedSearchCV
best_model = search.best_estimator_.named_steps['models']

# Предобработка данных через пайплайн без конечной модели
preprocessor = search.best_estimator_.named_steps['preprocessor']
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

all_feature_names = preprocessor.get_feature_names_out()

# Создаем DataFrame с соответствующими именами колонок
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, 
                                       columns=all_feature_names)
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, 
                                      columns=all_feature_names)

# Семпл данных для KernelExplainer
X_train_preprocessed_smpl = shap.sample(X_train_preprocessed_df, 
                                        samples, random_state=RANDOM_STATE)
X_test_preprocessed_smpl = shap.sample(X_test_preprocessed_df, 
                                       samples, random_state=RANDOM_STATE)

# Теперь, когда у нас есть DataFrame с именами признаков, мы можем использовать KernelExplainer
###explainer = shap.KernelExplainer(best_model.predict_proba, X_train_preprocessed_smpl)
explainer = shap.KernelExplainer(best_model.predict_proba, X_train_preprocessed_smpl)
shap_values = explainer.shap_values(X_test_preprocessed_smpl)

# Создаем объект Explanation для первого КЛАССА.
shap_values_explanation = shap.Explanation(
    values=shap_values[:,:,1], 
    base_values=explainer.expected_value,
    data=X_test_preprocessed_smpl,
    feature_names=all_feature_names
)

shap.summary_plot(shap_values_explanation, plot_size=[15,6])
plt.show()
'''

"\nsamples = 20\n \n# Извлечение лучшей модели из результатов RandomizedSearchCV\nbest_model = search.best_estimator_.named_steps['models']\n\n# Предобработка данных через пайплайн без конечной модели\npreprocessor = search.best_estimator_.named_steps['preprocessor']\nX_train_preprocessed = preprocessor.transform(X_train)\nX_test_preprocessed = preprocessor.transform(X_test)\n\nall_feature_names = preprocessor.get_feature_names_out()\n\n# Создаем DataFrame с соответствующими именами колонок\nX_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, \n                                       columns=all_feature_names)\nX_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, \n                                      columns=all_feature_names)\n\n# Семпл данных для KernelExplainer\nX_train_preprocessed_smpl = shap.sample(X_train_preprocessed_df, \n                                        samples, random_state=RANDOM_STATE)\nX_test_preprocessed_smpl = shap.sample(X_test_preprocessed_df, \

In [86]:
'''
shap.summary_plot(shap_values_explanation, plot_size=[15,6], plot_type='bar')
plt.show()
'''

"\nshap.summary_plot(shap_values_explanation, plot_size=[15,6], plot_type='bar')\nplt.show()\n"

In [87]:
'''
shap.plots.waterfall(shap_values_explanation[0]);
shap.plots.waterfall(shap_values_explanation[1]);
plt.show()
'''

'\nshap.plots.waterfall(shap_values_explanation[0]);\nshap.plots.waterfall(shap_values_explanation[1]);\nplt.show()\n'

# Отчёт и выводы


- В соответствии с поставленной задачей(см. первый раздел)
    - была проведена подготовка данных и их разведочный анализ, в результате которых:
        - были исправлены имена полей, исправлены типы данных, удалены малополезные признаки (в т. ч. с учётом мультиколлинеарности), заполнены пропуски, удалены явные и неявные дубликаты, ...
    - была натренирована модель, предсказывающая ...
        - выбрана наиболее оптимальная модель - **...**, показавшая наилучшие результаты:
            - качество предсказания: ...
            - это удовлетворяет заданный критерий качества: ...

# Appendix - comments legend

These icons and [tags] were used to simplify visualization and searching for conclusions, open questions and subtasks, not to forget to finish/check something important. It's convenient to copy a cell from here and paste it into the right place, adding details and then find it by tags or visually.

👷🚩[todo] 

⬆ this task is to be done

👷✅ 

⬆ the task has been done

👷🚨[SOS] 

⬆ need help

👷🔔[reminder] 

⬆ some work to do or check later

⚠️[!] 

⬆ important constraints or facts

💡 [!] 

⬆ something interesting

🏭 [data transformation] this change ⤴ should be done before training/testing/inference

👷🚧🚧🚧🚧🚧 [in progress] 

⬆ in progress right now

🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧⬇️🚧

➡️ some code to be reworked ⬅️

🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧⬆️🚧